# UppASD bcc Fe: Temperature Sweep Tutorial

This notebook is a step-by-step tutorial for computing thermodynamic trends of bcc Fe across temperature and using them to build phase-diagram intuition.

## Scientific objective
We run the same model at multiple temperatures and compare observables sensitive to phase transitions.

## Learning goals
- Build a reusable bcc Fe model template
- Configure temperature-aware initial-phase syntax (`ip_nphase` / `ip_mcanneal` or `ip_temp` variants)
- Run an automated sweep with progress feedback
- Interpret `m(T)`, Binder cumulant, susceptibility, and specific heat

## Recommended execution order
Run all cells in order. Start with a coarse grid, then refine near candidate transition regions.

## Units and conventions used in this tutorial

Unless explicitly stated otherwise, this notebook uses UppASD default output conventions:
- **Time**: seconds (`s`)
- **Temperature**: kelvin (`K`)
- **Energy**: milli-Rydberg (`mRy`)
- **Magnetization / moments**: Bohr magneton (`μ_B`)
- **Exchange parameters (`Jij`)**: `mRy`
- **DMI parameters (`Dij`)**: `mRy`
- **Binder cumulant**: dimensionless

In this temperature-sweep notebook, horizontal axes are temperature in `K`; energy-like outputs are treated as `mRy`.

In [ ]:
import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Install UppASD if missing in current kernel environment
if importlib.util.find_spec("uppasd") is None:
    print("Installing uppasd…")
    !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

# Core UppASD classes for model setup, execution, and results parsing
from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults

# Sweep helpers: temperature-token sync + automated run loop
from uppasd import set_temperature_token, validate_temperature_token, run_temperature_sweep

print("✅ UppASD imported successfully")

## Step 1 — Build the crystal and initial spin state

We define a minimal bcc unit-cell model to keep sweep runtime manageable while preserving the full workflow.

Key arrays:
- `cell`: lattice vectors
- `positions`: basis atom coordinates
- `species`: species labels
- `moments`: initial spin vectors (here aligned along +z)

In [ ]:
# Set up a single unit cell with one atom
a = 1.0
cell = np.array([[-a/2, a/2, a/2],[a/2, -a/2, a/2],[a/2, a/2, -a/2]])

positions = np.array([[0.0 , 0.0, 0.0]])
natom = positions.shape[0]
species = np.ones(natom, dtype=int)

moments = np.zeros((natom, 3))
moments[:, 2] = 1.0

system = SpinSystem(cell, positions, species, moments)


## Step 2 — Add exchange-shell interactions

`ExchangeShellTable` stores pair couplings shell-by-shell. This explicit setup makes parameter scans transparent and reproducible.

In this example, each `J_ij` value is paired with a representative bond vector in `r_ij` and added as a species-1 to species-1 interaction.

In [ ]:
exchange = ExchangeShellTable()

# Exchange couplings J_ij in mRy (shell-resolved)
J_ij = [1.33767,  0.7570, -0.05975, -0.0882]
r_ij = np.array([[a/2, a/2, a/2], [0, 0, a], [a, a, a/2], [3*a/2, a/2, a/2]])

for ij, jij in enumerate(J_ij):
    exchange.add_bond(1, 1, ij+1, r_ij[ij], jij)


## Step 3 — Configure a sweep-ready input template

The same template input is copied for every temperature point.

Important design principle: only temperature should vary between sweep points unless you are intentionally doing a multi-parameter study.

`set_temperature_token(...)` and `validate_temperature_token(...)` keep initial-phase and simulation temperatures synchronized across supported syntax conventions.

In [ ]:
# Build a reusable input template for each sweep point
inp = ASDInput()
Nx = 10
Ny = 10
Nz = 10
inp.block("system").set(ncell=(Nx, Ny, Nz), bc=("P", "P", "P"), do_prnstruct=2, sym=1)

# Initial phase setup (choose one syntax style)

# A) Schedule syntax
# inp.block("initial").set(ip_mode="S", ip_nphase=(1, '\n', 100, 0.0, 1e-15, 1.0), initmag=1)
# For MC / Heat bath schedule syntax use instead:
# inp.block("initial").set(ip_mode="H", ip_mcanneal=(1, '\n', 1000, 0.0), initmag=1)

# B) Scalar syntax (also supported by sweep helpers)
# LLG: ip_mode='S' with ip_temp + ip_nstep (+ optional ip_timestep/ip_damping)
# inp.block("initial").set(ip_mode="S", ip_temp=5.0, ip_nstep=100, ip_timestep=1e-15, ip_damping=1.0, initmag=1)
# MC/HB: ip_mode='M' or 'H' with ip_temp + ip_mcnstep
inp.block("initial").set(ip_mode="H", ip_temp=5.0, ip_mcnstep=5000, initmag=1)

# Main simulation settings
inp.block("simulation").set(mode="S", nstep=5000, timestep=1e-15, damping=0.5, temperature=5.0)

# Output settings needed for thermodynamic analysis
inp.block("measurables").set(plotenergy=1, do_avrg="Y", do_cumu="Y")

# Synchronize initial-phase and simulation temperature tokens
set_temperature_token(inp, 5.0)
print(validate_temperature_token(inp))

## Run the temperature sweep

`run_temperature_sweep(...)` processes one temperature at a time and prints progress in the notebook output.

Tip: start with a small temperature grid for quick testing, then refine near the transition region.

In [ ]:
# Temperature sweep example using built-in helper (with progress output)
temperatures = np.linspace(100.0, 1500.0, 15)

sweep_results = run_temperature_sweep(
    workdir_root="bccFe_sweep",
    temperatures=temperatures,
    system=system,
    inp=inp,
    exchange=exchange,
    dmi=None,
    clean=True,
    verbose=True,
 )

sweep_df = pd.DataFrame(sweep_results)
display(sweep_df)

## Step 5 — Analyze thermodynamic curves and estimate transition temperatures

The plots below provide complementary transition indicators:
- `m(T)`: order parameter decay with temperature
- Binder cumulant: finite-size-sensitive shape/crossing behavior
- `chi(T)`: magnetic fluctuation peak
- `cv(T)`: energy fluctuation peak
- `dE/dT`: additional numerical trend from energy curve

### Interpretation notes
- Peak positions from `chi` and `cv` are useful first estimates of transition windows.
- For publishable phase boundaries, repeat for larger system sizes and perform finite-size scaling.

In [ ]:
# Phase-diagram diagnostics from temperature sweep (units: T[K], m[μ_B], E[mRy])
required = ["temperature", "m", "binder", "chi", "cv", "energy"]
missing = [col for col in required if col not in sweep_df.columns]
if missing:
    raise ValueError(f"Missing columns in sweep_df: {missing}")

plot_df = sweep_df.dropna(subset=required).sort_values("temperature").reset_index(drop=True)
if plot_df.empty:
    raise ValueError("No valid sweep rows to plot")

fig, axes = plt.subplots(2, 3, figsize=(14, 8), constrained_layout=True)
axes = axes.ravel()

axes[0].plot(plot_df["temperature"], plot_df["m"], marker="o")
axes[0].set_title("Magnetization")
axes[0].set_xlabel("Temperature [K]")
axes[0].set_ylabel("m [μ_B]")

axes[1].plot(plot_df["temperature"], plot_df["binder"], marker="o")
axes[1].set_title("Binder Cumulant")
axes[1].set_xlabel("Temperature [K]")
axes[1].set_ylabel("U4 [dimensionless]")

axes[2].plot(plot_df["temperature"], plot_df["chi"], marker="o")
axes[2].set_title("Susceptibility")
axes[2].set_xlabel("Temperature [K]")
axes[2].set_ylabel("chi [UppASD units]")

axes[3].plot(plot_df["temperature"], plot_df["cv"], marker="o")
axes[3].set_title("Specific Heat")
axes[3].set_xlabel("Temperature [K]")
axes[3].set_ylabel("cv [UppASD units]")

axes[4].plot(plot_df["temperature"], plot_df["energy"], marker="o")
axes[4].set_title("Energy")
axes[4].set_xlabel("Temperature [K]")
axes[4].set_ylabel("E [mRy]")

# Optional numerical derivative dE/dT as additional transition indicator
dEdT = np.gradient(plot_df["energy"].to_numpy(), plot_df["temperature"].to_numpy())
axes[5].plot(plot_df["temperature"], dEdT, marker="o")
axes[5].set_title("dE/dT")
axes[5].set_xlabel("Temperature [K]")
axes[5].set_ylabel("dE/dT [mRy/K]")

plt.show()

# Simple transition-temperature indicators
tc_chi = float(plot_df.loc[plot_df["chi"].idxmax(), "temperature"])
tc_cv = float(plot_df.loc[plot_df["cv"].idxmax(), "temperature"])
print(f"Estimated Tc from chi peak: {tc_chi:.3f} K")
print(f"Estimated Tc from cv peak : {tc_cv:.3f} K")